In [1]:
import os 
import sys 

current_dir = os.getcwd()
dir_to_use = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.append(dir_to_use)

In [4]:
from premsql.datasets import Text2SQLDataset

bird_dataset = Text2SQLDataset(
    dataset_name='bird', split="train", force_download=False,
    dataset_folder="../data"
).setup_dataset(
    num_fewshot=3,
    num_rows=3,
    model_name_or_path="premai-io/prem-1B-SQL",
)

2024-09-04 12:27:46,035 - [BIRD-DATASET] - INFO - Loaded Bird Dataset
2024-09-04 12:27:46,036 - [BIRD-DATASET] - INFO - Setting up Bird Dataset
Applying prompt: 100%|██████████| 3/3 [00:00<00:00, 2407.75it/s]
2024-09-04 12:27:46,846 - [DATASET] - INFO - Casted dataset with model chat template
2024-09-04 12:27:46,846 - [DATASET] - INFO - Starting Tokenization ...
Tokenizing: 100%|██████████| 3/3 [00:00<00:00, 332.56it/s]


In [6]:
sample = bird_dataset[0]["raw"]["prompt"]

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("premai-io/prem-1B-SQL")
tokenizer = AutoTokenizer.from_pretrained(
    "premai-io/prem-1B-SQL",
    padding_side="right"
)

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}


model.safetensors.index.json:   0%|          | 0.00/18.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.huggingface.co/repos/71/6e/716e940d175ff86a403807ac7370634f8bfe1648f58fb614473f5bc8b410f2ef/34a7e0c12737c54f1c824032489551825c626a1643c93ff05baa82cd588d83ab?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00001-of-00002.safetensors%3B+filename%3D%22model-00001-of-00002.safetensors%22%3B&Expires=1725692336&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNTY5MjMzNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzcxLzZlLzcxNmU5NDBkMTc1ZmY4NmE0MDM4MDdhYzczNzA2MzRmOGJmZTE2NDhmNThmYjYxNDQ3M2Y1YmM4YjQxMGYyZWYvMzRhN2UwYzEyNzM3YzU0ZjFjODI0MDMyNDg5NTUxODI1YzYyNmExNjQzYzkzZmYwNWJhYTgyY2Q1ODhkODNhYj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=rV6koVhoxt%7E36dbs99r8yfGcopfU8V4SMwokXvagTjKGPHgjn97-xgb1PbZatajmFbTxei3b17aYuOduWfTiqZdSGgfGVjY43KObAve0dTC1H4h7yTMRXjTP3iDUtsaN0%7EV8FykFDyGMslM%7ESWYs6jXRmlUDTcQXKawAyFV7vRoKiW80Ff7zgZ1T6qjEJJ4uteoSMS-9Doj

╭─────────────────────────────────────── Traceback (most recent call last) ───────────────────────────────────────╮
│ /home/anindya/miniconda3/envs/deep/lib/python3.10/site-packages/urllib3/response.py:748 in _error_catcher       │
│                                                                                                                 │
│    745 │   │                                                                                                    │
│    746 │   │   try:                                                                                             │
│    747 │   │   │   try:                                                                                         │
│ ❱  748 │   │   │   │   yield                                                                                    │
│    749 │   │   │                                                                                                │
│    750 │   │   │   except SocketTimeout as e:                                                                   │
│    751 │   │   │   │   # FIXME: Ideally we'd like to include the url in the ReadTimeoutError bu                 │
│                                                                                                                 │
│ /home/anindya/miniconda3/envs/deep/lib/python3.10/site-packages/urllib3/response.py:873 in _raw_read            │
│                                                                                                                 │
│    870 │   │   fp_closed = getattr(self._fp, "closed", False)                                                   │
│    871 │   │                                                                                                    │
│    872 │   │   with self._error_catcher():                                                                      │
│ ❱  873 │   │   │   data = self._fp_read(amt, read1=read1) if not fp_closed else b""                             │
│    874 │   │   │   if amt is not None and amt != 0 and not data:                                                │
│    875 │   │   │   │   # Platform-specific: Buggy versions of Python.                                           │
│    876 │   │   │   │   # Close the connection when no data is returned                                          │
│                                                                                                                 │
│ /home/anindya/miniconda3/envs/deep/lib/python3.10/site-packages/urllib3/response.py:856 in _fp_read             │
│                                                                                                                 │
│    853 │   │   │   return self._fp.read1(amt) if amt is not None else self._fp.read1()                          │
│    854 │   │   else:                                                                                            │
│    855 │   │   │   # StringIO doesn't like amt=None                                                             │
│ ❱  856 │   │   │   return self._fp.read(amt) if amt is not None else self._fp.read()                            │
│    857 │                                                                                                        │
│    858 │   def _raw_read(                                                                                       │
│    859 │   │   self,                                                                                            │
│                                                                                                                 │
│ /home/anindya/miniconda3/envs/deep/lib/python3.10/http/client.py:466 in read                                    │
│                                                                                                                 │
│    463 │   │   │   if self.length is not None and amt > self.length:                                            │
│    464 │   │   │   │   # clip the read to the "end of 